A. Enfoque de validación
Utilicen la base respondieron. Para cada año, dividan las observaciones en
una base de prueba (test) y una de entrenamiento (train) utilizando el
comando train_test_split. La base de entrenamiento debe comprender el
70% de los datos, y la semilla a utilizar (random state instance) debe ser 444.
Establezca a desocupado como su variable dependiente en la base de
entrenamiento (vector y). El resto de las variables seleccionadas serán las
variables independientes (matriz X). Recuerden agregar la columna de unos
(1).
1. Cree una tabla de diferencia de medias entre la base de entrenamiento
y la de testeo de las características seleccionadas en su matriz X.
Comente la tabla de la diferencia de medias de sus variables entre
entrenamiento y testeo.

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Cargar el archivo Excel
respondieron = pd.read_excel('respondieron.xlsx')

# Filtrar años 2004 y 2024
respondieron = respondieron[respondieron['AÑO'].isin([2004, 2024])]

# Crear la variable binaria 'desocupado'
respondieron['desocupado'] = (respondieron['ESTADO'] == 2).astype(int)

# Variables independientes
variables_x = ['P21', 'ingreso_ajustado', 'salario_semanal', 'edad', 'horastrab', 'edad2', 'educ']

# Recorrer por año
for anio in [2004, 2024]:
    datos = respondieron[respondieron['AÑO'] == anio].copy()
    
    # Crear X e y
    X = datos[variables_x].copy()
    X['constante'] = 1
    y = datos['desocupado']
    
    # Combinar X + y + año
    data = X.copy()
    data['desocupado'] = y
    data['AÑO'] = anio  # conservar la columna de año
    
    # Dividir en entrenamiento y prueba
    train, test = train_test_split(data, test_size=0.3, random_state=444)
    
    # Guardar a archivos Excel
    train.to_excel(f'base_entrenamiento_{anio}.xlsx', index=False)
    test.to_excel(f'base_prueba_{anio}.xlsx', index=False)



In [11]:
import pandas as pd

# Variables independientes (matriz X)
variables_x = ['P21', 'ingreso_ajustado', 'salario_semanal', 'edad',
               'horastrab', 'edad2', 'educ', 'constante']

# Función para calcular tabla de diferencia de medias entre dos archivos
def comparar_medias(archivo_2004, archivo_2024, variables):
    # Leer archivos
    df_2004 = pd.read_excel(archivo_2004)
    df_2024 = pd.read_excel(archivo_2024)

    # Calcular medias
    mean_2004 = df_2004[variables].mean()
    mean_2024 = df_2024[variables].mean()
    diff_abs = (mean_2004 - mean_2024).abs()

    # Tabla resumen
    tabla = pd.DataFrame({
        'Media 2004': mean_2004,
        'Media 2024': mean_2024,
        'Diferencia Absoluta': diff_abs
    })

    return tabla

# Comparar entrenamiento 2004 vs 2024
tabla_diff_train = comparar_medias('base_entrenamiento_2004.xlsx', 'base_entrenamiento_2024.xlsx', variables_x)
print("📊 Diferencias de medias entre bases de ENTRENAMIENTO (2004 vs 2024):")
print(tabla_diff_train, "\n")

# Comparar prueba 2004 vs 2024
tabla_diff_test = comparar_medias('base_prueba_2004.xlsx', 'base_prueba_2024.xlsx', variables_x)
print("📊 Diferencias de medias entre bases de PRUEBA (2004 vs 2024):")
print(tabla_diff_test)


📊 Diferencias de medias entre bases de ENTRENAMIENTO (2004 vs 2024):
                    Media 2004     Media 2024  Diferencia Absoluta
P21                 700.654373  352600.057790        351899.403417
ingreso_ajustado  36052.788988  352600.057790        316547.268802
salario_semanal     901.319725    8815.001445          7913.681720
edad                 38.169652      40.724646             2.554994
horastrab            35.879874      35.190935             0.688939
edad2              1600.306639    1806.294051           205.987412
educ                  5.384181       5.283333             0.100847
constante             1.000000       1.000000             0.000000 

📊 Diferencias de medias entre bases de PRUEBA (2004 vs 2024):
                    Media 2004     Media 2024  Diferencia Absoluta
P21                 814.776961  339880.916777        339066.139816
ingreso_ajustado  41925.067438  339880.916777        297955.849339
salario_semanal    1048.126686    8497.022919          7448.896

Como podemos observar los datos entre la base de entrenamiento de 2004 y la de prueba del mismo año son levemente distintas entre sí en los ingresos habituales (P21), por lo tanto los ingresos ajustados y el salario semanal van a ser distintos en la misma escala a los valores de 2024. En cambio P21, cambia muy poco en la base de entrenamiento de 2024 con la de prueba de 2024 por lo que podemos afirmar que por los casos tomados en la semilla 444 la varianza en el 2004 es mayor a la de 2024 en el ingreso. Esto en muy pequeña medida sugiere mayor igualdad de ingresos. De todas formas los patrones son similares entre entrenamiento y prueba, más que nada en las otras variables no mencionadas. 
Entre el 2004 y el 2024 podemos observar claramente las consecuencias inflacionarias ya que el ingreso subió más de 40 veces su valor y el ingreso real cayó casi un 20% teniendo en cuenta la base de prueba (la más real). La edad media de trabajo aumentó mas de dos años. Si bien puede haber sido por muchas razones creemos que esto es principalmente debido al aumento de la inmigración interna de jóvenes del interior buscando nuevas oportunidades en Buenos Aires u otros lugares más desarrollados durante esas décadas. Las horas trabajadas disminuyeron en promedio más de una hora quizás debido a las nuevas tecnologías que ahorran tiempo. Por ejemplo un celular para comunicarse más rápido. La educación casi no varía realmente. 
Por último la variable constante o intercepto ayuda a los modelos a ser más flexibles y ajustarse mejor a los datos ya que no tiene que pasar por el punto de origen.
